In [1]:
# Data exploration
import pandas as pd

# Numerical
import numpy as np

In [2]:
#import the functions from their corresponding files
from NYC_GetCleaned_PrecinctData import getPrecinctData
from data_utility import filterData

0.0
0.0
0.0
0.0
0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5835784 entries, 1 to 6039728
Data columns (total 16 columns):
CMPLNT_FR_DT         object
CMPLNT_FR_TM         object
CMPLNT_TO_DT         object
CMPLNT_TO_TM         object
RPT_DT               object
OFNS_DESC            object
PD_DESC              object
CRM_ATPT_CPTD_CD     object
LAW_CAT_CD           object
BORO_NM              object
ADDR_PCT_CD          float64
LOC_OF_OCCUR_DESC    object
PREM_TYP_DESC        object
Latitude             float64
Longitude            float64
Lat_Lon              object
dtypes: float64(3), object(13)
memory usage: 756.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 4 columns):
Shape_Area    77 non-null float64
Shape_Leng    77 non-null float64
Precinct      77 non-null int64
the_geom      77 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 2.5+ KB


In [3]:
#Get cleaned data from NYC_GetCleaned_HistoricData
crimes_original = getPrecinctData()
crimes_original

,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,OFNS_DESC,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,Latitude,Longitude,Lat_Lon,Shape_Area,Shape_Leng,Precinct,the_geom
0,3/30/2014,20:40:00,3/30/2014,20:50:00,3/30/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,RESIDENCE - APT. HOUSE,40.836162,-73.915250,"(40.836161968, -73.915249813)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
1,4/1/2014,15:00:00,4/1/2014,15:05:00,4/1/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.825000,-73.916615,"(40.825000207, -73.916615377)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2,4/6/2014,14:45:00,4/6/2014,14:45:00,4/6/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.830013,-73.918112,"(40.830013132, -73.918112312)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
3,5/6/2014,14:00:00,5/6/2014,14:05:00,5/6/2014,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.828608,-73.921742,"(40.828607613, -73.92174192)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
4,3/6/2014,23:42:00,3/6/2014,23:42:00,3/7/2014,POSSESSION OF STOLEN PROPERTY,"STOLEN PROPERTY 2,1,POSSESSION",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.838752,-73.913758,"(40.83875187, -73.913757556)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
5,4/2/2014,00:45:00,4/24/2014,00:55:00,4/24/2014,ROBBERY,"ROBBERY,PERSONAL ELECTRONIC DEVICE",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,OPEN AREAS (OPEN LOTS),40.835405,-73.920306,"(40.835405287, -73.920306476)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
6,4/4/2014,15:50:00,4/4/2014,16:05:00,4/4/2014,ROBBERY,"ROBBERY,PERSONAL ELECTRONIC DEVICE",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,PARK/PLAYGROUND,40.841263,-73.909396,"(40.841262681, -73.909395681)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
7,2/13/2014,12:45:00,2/13/2014,12:55:00,2/13/2014,ROBBERY,"ROBBERY,PERSONAL ELECTRONIC DEVICE",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.834171,-73.922220,"(40.834171472, -73.922219615)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
8,3/15/2014,15:30:00,3/15/2014,15:40:00,3/15/2014,ROBBERY,"ROBBERY,OPEN AREA UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.838370,-73.912710,"(40.838369567, -73.912710005)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
9,4/26/2014,10:12:00,4/26/2014,10:18:00,4/26/2014,ROBBERY,"ROBBERY,PERSONAL ELECTRONIC DEVICE",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.826933,-73.913632,"(40.826933037, -73.913631954)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...


In [4]:
"""Create a datetime index of times that crimes were reported to have been committed"""

dfCopy = crimes_original.copy()
def eliminate_dates(x):
    if x[2] > '2050':
        x = None
    elif x[2] < '2010':
        x = None
    else:
        aa= '/'.join(x)
        return (aa)

#get dummy columns for crime categories
#dfCopy = dfCopy.join(dfCopy['LAW_CAT_CD'].str.get_dummies())

#dfCopy = dfCopy.join(dfCopy['BORO_NM'].str.get_dummies())

#Create index with DateTime
dfCopy['CMPLNT_FR_DT'] = dfCopy['CMPLNT_FR_DT'].str.split("/")
dfCopy['CMPLNT_FR_DT'] = dfCopy['CMPLNT_FR_DT'].apply(lambda x: eliminate_dates(x))

#Combing date and time columns
dfCopy['StartTime'] = dfCopy['CMPLNT_FR_DT'] +' '+dfCopy['CMPLNT_FR_TM']
dfCopy['StartTime'] = pd.to_datetime(dfCopy['StartTime'])


#set full date as index
#dfCopy.set_index('StartTime', inplace=True)        



In [12]:
dfCopy['Time_Day'] = 'none'


In [13]:
dfCopy.head()

,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,OFNS_DESC,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,...,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,Latitude,Longitude,Lat_Lon,Shape_Area,Shape_Leng,Precinct,the_geom,Time_Day
StartTime,,,,,,,,,,,,,,,,,,,,,
2014-03-30 20:40:00,3/30/2014,20:40:00,3/30/2014,20:50:00,3/30/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",ATTEMPTED,FELONY,BRONX,...,INSIDE,RESIDENCE - APT. HOUSE,40.836162,-73.915250,"(40.836161968, -73.915249813)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...,none
2014-04-01 15:00:00,4/1/2014,15:00:00,4/1/2014,15:05:00,4/1/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,...,INSIDE,STREET,40.825000,-73.916615,"(40.825000207, -73.916615377)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...,none
2014-04-06 14:45:00,4/6/2014,14:45:00,4/6/2014,14:45:00,4/6/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,...,INSIDE,STREET,40.830013,-73.918112,"(40.830013132, -73.918112312)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...,none
2014-05-06 14:00:00,5/6/2014,14:00:00,5/6/2014,14:05:00,5/6/2014,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",ATTEMPTED,FELONY,BRONX,...,INSIDE,STREET,40.828608,-73.921742,"(40.828607613, -73.92174192)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...,none
2014-03-06 23:42:00,3/6/2014,23:42:00,3/6/2014,23:42:00,3/7/2014,POSSESSION OF STOLEN PROPERTY,"STOLEN PROPERTY 2,1,POSSESSION",ATTEMPTED,FELONY,BRONX,...,INSIDE,STREET,40.838752,-73.913758,"(40.83875187, -73.913757556)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...,none


In [14]:
if dfCopy['StartTime'] == '20':
    dfCopy['Time_Day'] = 'Evening'

KeyError: 'StartTime'

In [6]:
data = dfCopy.drop(['CMPLNT_FR_DT','CMPLNT_FR_TM','CMPLNT_TO_DT','CMPLNT_TO_TM','RPT_DT','ADDR_PCT_CD','Latitude','Longitude','Lat_Lon','Shape_Area','Shape_Leng','the_geom'],axis=1)

In [7]:
filter_data = filterData(data,'Precinct',44)
filter_data

,OFNS_DESC,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,Precinct
StartTime,,,,,,,,
2014-03-30 20:40:00,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",ATTEMPTED,FELONY,BRONX,INSIDE,RESIDENCE - APT. HOUSE,44
2014-04-01 15:00:00,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,INSIDE,STREET,44
2014-04-06 14:45:00,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,INSIDE,STREET,44
2014-05-06 14:00:00,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",ATTEMPTED,FELONY,BRONX,INSIDE,STREET,44
2014-03-06 23:42:00,POSSESSION OF STOLEN PROPERTY,"STOLEN PROPERTY 2,1,POSSESSION",ATTEMPTED,FELONY,BRONX,INSIDE,STREET,44
2014-04-02 00:45:00,ROBBERY,"ROBBERY,PERSONAL ELECTRONIC DEVICE",ATTEMPTED,FELONY,BRONX,INSIDE,OPEN AREAS (OPEN LOTS),44
2014-04-04 15:50:00,ROBBERY,"ROBBERY,PERSONAL ELECTRONIC DEVICE",ATTEMPTED,FELONY,BRONX,INSIDE,PARK/PLAYGROUND,44
2014-02-13 12:45:00,ROBBERY,"ROBBERY,PERSONAL ELECTRONIC DEVICE",ATTEMPTED,FELONY,BRONX,INSIDE,STREET,44
2014-03-15 15:30:00,ROBBERY,"ROBBERY,OPEN AREA UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,INSIDE,STREET,44


In [10]:
dfCopy_copy = dfCopy.copy()

In [11]:
dfCopy_copy.head()

,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,OFNS_DESC,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,Latitude,Longitude,Lat_Lon,Shape_Area,Shape_Leng,Precinct,the_geom
StartTime,,,,,,,,,,,,,,,,,,,,
2014-03-30 20:40:00,3/30/2014,20:40:00,3/30/2014,20:50:00,3/30/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,RESIDENCE - APT. HOUSE,40.836162,-73.915250,"(40.836161968, -73.915249813)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2014-04-01 15:00:00,4/1/2014,15:00:00,4/1/2014,15:05:00,4/1/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.825000,-73.916615,"(40.825000207, -73.916615377)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2014-04-06 14:45:00,4/6/2014,14:45:00,4/6/2014,14:45:00,4/6/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.830013,-73.918112,"(40.830013132, -73.918112312)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2014-05-06 14:00:00,5/6/2014,14:00:00,5/6/2014,14:05:00,5/6/2014,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.828608,-73.921742,"(40.828607613, -73.92174192)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
2014-03-06 23:42:00,3/6/2014,23:42:00,3/6/2014,23:42:00,3/7/2014,POSSESSION OF STOLEN PROPERTY,"STOLEN PROPERTY 2,1,POSSESSION",ATTEMPTED,FELONY,BRONX,44.0,INSIDE,STREET,40.838752,-73.913758,"(40.83875187, -73.913757556)",5.553055e+07,31394.624816,44,MULTIPOLYGON (((-73.90270221290083 40.84458296...
